In [77]:
import pandas as pd
import matplotlib as plt
import statsmodels as sm
import numpy as np

In [147]:
df = pd.read_csv("data/inf_exp_data.csv")
df

/Users/duncanblack/opt/anaconda3/envs/Inflation/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (5,6,13,15,16,18,32,34,36,37,39,40,41,45,46,47,49,50,51,52,53,54,55,71,80,81,82,83,84,85,86,87,88,89,90,91,92,96,98,100,101,103,104,105,106,107,108,109) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,CASEID,YYYYMM,YYYYQ,YYYY,ID,IDPREV,DATEPR,ICS,ICC,ICE,...,GAS1PX1,GAS1PX2,GAS1,PINC,PINC2,PJOB,PSSA,PCRY,PSTK,WT
0,1,197801,19781,1978,1,,,76.71,116.23,51.32,...,,,,,,,,,,0.00
1,2,197801,19781,1978,2,,,121.12,154.08,99.94,...,,,,,,,,,,0.72
2,3,197801,19781,1978,3,,,2.70,2.70,2.70,...,,,,,,,,,,1.03
3,4,197801,19781,1978,4,,,76.71,154.08,27.01,...,,,,,,,,,,0.00
4,5,197801,19781,1978,5,,,121.12,116.23,124.25,...,,,,,,,,,,0.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303105,303106,202203,20221,2022,2096,1100,202109,31.60,77.69,2.00,...,1,50,50,5.0,30.0,5.0,15.0,3,60.0,0.50
303106,303107,202203,20221,2022,2097,1159,202109,61.21,115.53,26.31,...,3,,0,70.0,70.0,25.0,5.0,3,15.0,0.50
303107,303108,202203,20221,2022,2098,1024,202109,31.60,39.84,26.31,...,1,15,15,40.0,0.0,20.0,40.0,5,50.0,0.50
303108,303109,202203,20221,2022,2099,1094,202109,31.60,77.69,2.00,...,1,100,100,0.0,0.0,0.0,25.0,5,50.0,0.50


In [4]:
colnames=df.columns

In [5]:
colnames

Index(['CASEID', 'YYYYMM', 'YYYYQ', 'YYYY', 'ID', 'IDPREV', 'DATEPR', 'ICS',
       'ICC', 'ICE',
       ...
       'GAS1PX1', 'GAS1PX2', 'GAS1', 'PINC', 'PINC2', 'PJOB', 'PSSA', 'PCRY',
       'PSTK', 'WT'],
      dtype='object', length=111)

In [148]:
#Period between 1984:1 to 2012:12
cleaned_data=df[(df["YYYYMM"] >= 198401)&(df["YYYYMM"] <= 201212)]
#Only keep first time responces
cleaned_data=cleaned_data[cleaned_data["DATEPR"] == "      "]

In [114]:
# Questions Q1,Q2

questions=["DUR","PX1Q2"]

cleaned_data[questions]

,DUR,PX1Q2
57768,1,1
57769,1,98
57770,1,5
57771,9,4
57772,1,10
...,...,...
239713,1,5
239714,9,
239715,1,10
239716,1,4


In [9]:
#Other non controls

other = ["RATEX","BUS12","BUS5","UNEMP","GOVT",]

cleaned_data[other]

,RATEX,BUS12,BUS5,UNEMP,GOVT
57768,5,2,4,3,5
57769,5,1,1,5,1
57770,3,1,1,3,3
57771,1,9,98,1,1
57772,1,1,1,3,3
...,...,...,...,...,...
239713,1,1,2,5,3
239714,3,1,1,1,3
239715,5,8,5,5,3
239716,1,5,5,3,5


In [8]:
#Control
control = ["AGE","SEX","MARRY","EDUC","REGION","YYYYMM","RINC","NUMADT","NUMKID"]

cleaned_data[control]

,AGE,SEX,MARRY,EDUC,REGION,YYYYMM,RINC,NUMADT,NUMKID
57768,70,2,4,3,4,198401,3,2,1
57769,69,2,1,3,1,198401,8,2,0
57770,35,2,1,6,3,198401,5,2,1
57771,50,1,1,1,4,198401,5,2,2
57772,32,2,1,3,3,198401,3,2,2
...,...,...,...,...,...,...,...,...,...
239713,19,1,5,4,4,201212,1,2,0
239714,30,2,1,4,1,201212,3,2,2
239715,26,2,5,6,4,201212,5,2,0
239716,40,2,1,5,3,201212,3,2,2


In [149]:
#Transform Variables

#AGE
cleaned_data["AGE"] = cleaned_data["AGE"].replace({'  ':0})
cleaned_data = cleaned_data.astype({"AGE": int})
cleaned_data["AGE2"] = cleaned_data["AGE"]*cleaned_data["AGE"]
cleaned_data["AGE3"] = cleaned_data["AGE2"]*cleaned_data["AGE"]

#SEX
cleaned_data = cleaned_data.astype({"SEX": str})
cleaned_data["SEX"] = cleaned_data["SEX"].replace({"1":"0","2":"1"})

#MARRY
cleaned_data = cleaned_data.astype({"MARRY": str})
cleaned_data["MARRY"] = cleaned_data["MARRY"].replace({"2":"0","3":"0","4":"0","5":"0"})

#COLLEGE
cleaned_data = cleaned_data.astype({"EDUC": str})
cleaned_data["COLLEGE"] = cleaned_data["EDUC"].replace({"1":"0","2":"0","3":"0","4":"0","5":"1","6":"1"})

#REGION DUMMIES
cleaned_data = cleaned_data.astype({"REGION": str})
cleaned_data["REGION"] = cleaned_data["REGION"].replace({"1":"WEST","2":"NORTH CENTRAL","3":"NORTHEAST","4":"SOUTH"})
region=cleaned_data["REGION"]
region_dummies=pd.get_dummies(region)
region_dummies=region_dummies.drop([' '],axis=1)

#MONTHY DUMMIES
cleaned_data = cleaned_data.astype({"YYYYMM": str})
months=cleaned_data["YYYYMM"].str[4:]
month_dummies=pd.get_dummies(months)


#LOG INCOME
cleaned_data["LOG_RINC"] = np.log(cleaned_data["RINC"])

#FAMILY SIZE
cleaned_data["NUMADT"]=cleaned_data["NUMADT"].replace({' ':0})
cleaned_data["NUMKID"]=cleaned_data["NUMKID"].replace({' ':0})
cleaned_data = cleaned_data.astype({"NUMADT": int})
cleaned_data = cleaned_data.astype({"NUMKID": int})
cleaned_data["FAMILY_SIZE"] = cleaned_data["NUMADT"]+cleaned_data["NUMKID"]

#DLZ
cleaned_data["DLZ"] = cleaned_data["ID"]*0
cleaned_data["DLZ"][(df["YYYYMM"] >= 200812)&(df["YYYYMM"] <= 201212)] = 1

/Users/duncanblack/opt/anaconda3/envs/Inflation/lib/python3.7/site-packages/ipykernel_launcher.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  
/Users/duncanblack/opt/anaconda3/envs/Inflation/lib/python3.7/site-packages/ipykernel_launcher.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  import sys
/Users/duncanblack/opt/anaconda3/envs/Inflation/lib/python3.7/site-packages/ipykernel_launcher.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which 

In [180]:
#Summary Stats

cleaned_data = pd.concat([cleaned_data,region_dummies],axis=1)
stats_noncat = ["AGE","AGE2","AGE3","LOG_RINC","FAMILY_SIZE"]
stats_cat = ["SEX","MARRY","COLLEGE","DLZ","NORTH CENTRAL","NORTHEAST","SOUTH","WEST"]

cleaned_data[stats_cat] = cleaned_data[stats_cat].astype(dtype="category")
cleaned_data[stats_noncat].describe()
cleaned_data[stats_cat].describe()

cleaned_data["DUR"] = cleaned_data["DUR"].astype(dtype="category")
cleaned_data["PX1Q2"] = cleaned_data["PX1Q2"].replace({'  ':0})
cleaned_data["PX1Q2"] = cleaned_data["PX1Q2"].astype({"PX1Q2": int})

cleaned_data[["PX1Q2"]].describe()


,PX1Q2
count,105725.000000
mean,12.700979
std,27.124576
min,0.000000
25%,2.000000
50%,3.000000
75%,7.000000
max,99.000000


In [173]:
cleaned_data[stats_cat].describe()

,SEX,MARRY,COLLEGE,DLZ,NORTH CENTRAL,NORTHEAST,SOUTH,WEST
count,105725,105725,105725,105725,105725,105725,105725,105725
unique,3,3,3,2,2,2,2,2
top,1,1,0,0,0,0,0,0
freq,58830,61361,68306,91056,77501,85542,69786,84354


In [174]:
cleaned_data[stats_noncat].describe()

,AGE,AGE2,AGE3,LOG_RINC,FAMILY_SIZE
count,105725.000000,105725.000000,105725.000000,105725.000000,105725.000000
mean,47.074476,2533.398146,150545.770584,1.098606,2.585841
std,17.815578,1793.995846,152796.598422,0.590766,1.400011
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,33.000000,1089.000000,35937.000000,1.098612,2.000000
50%,45.000000,2025.000000,91125.000000,1.098612,2.000000
75%,60.000000,3600.000000,216000.000000,1.609438,4.000000
max,97.000000,9409.000000,912673.000000,2.197225,10.000000


In [178]:
cleaned_data[["DUR"]].describe()

,DUR
count,105725
unique,5
top,1
freq,72115


In [182]:
cleaned_data["MARRY"]

57768     0
57769     1
57770     1
57771     1
57772     1
         ..
239713    0
239714    1
239715    0
239716    1
239717    0
Name: MARRY, Length: 105725, dtype: category
Categories (3, object): [' ', '0', '1']